In [ ]:
##############################################################################
file_name = "./data/my_file"


def process_file(fd):
    pass


##############################################################################

fd = open(file_name)
try:
    process_file(fd)
finally:
    fd.close()

##############################################################################
with open(file_name) as fd:
    process_file(fd)
    

In [ ]:
# context manager
import contextlib


def stop_datebase():
    pass


def start_datebase():
    pass


##############################################################################
@contextlib.contextmanager
def db_handler():
    try:
        stop_datebase()
        yield
    finally:
        start_datebase()


##############################################################################
def run():
    pass


class dbhandler_decorator(contextlib.ContextDecorator):
    def __enter__(self):
        stop_datebase()
        return self

    def __exit__(self, ext_type, ex_value, ex_traceback):
        start_datebase()


@dbhandler_decorator
def offline_backup():
    run("pg_dump database")

In [ ]:
# comprehensions
import re
from typing import Iterable, Set


def ARN_REGEX():
    pass


##############################################################################
def collect_account_ids_from_arns(arns: Iterable[str]) -> Set[str]:
    """
    arn:partition:region:account-id:resource_id 형태의 ARN들이 주어진 경우
    고유한 계정 ID (account-id)를 찾아서 반환
    """
    collected_account_ids = set()
    for arn in arns:
        matched = re.match(ARN_REGEX, arn)
        if matched is not None:
            account_id = matched.groupdict()["account_id"]
            collected_account_ids.add(account_id)
    return collected_account_ids


##############################################################################
def collect_account_ids_from_arns(arns: Iterable[str]) -> Set[str]:
    matched_arns = filter(None, (re.match(ARN_REGEX, arn) for arn in arns))
    return {m.groupdict()["account_id"] for m in matched_arns}


##############################################################################
def collect_account_ids_from_arns(arns: Iterable[str]) -> Set[str]:
    return {
        matched.groupdict()['account_id']
        for arn in arns
        if (matched := re.match(ARN_REGEX,arn)) is not None
    }

In [44]:
# under bar
##############################################################################
class Connector:
    def __init__(self, source) -> None:
        self.source = source
        self._timeout = 60


connector = Connector("postgresql://localhost")

print(connector.source)
print(connector._timeout)
print(connector.__dict__)


##############################################################################
class Connector:
    def __init__(self, source) -> None:
        self.source = source
        self.__timeout = 60


connector = Connector("postgresql://localhost")

print(connector.source)
print(connector.__timeout)
print(connector.__dict__)


##############################################################################
class Connector:
    def __init__(self, source) -> None:
        self.source = source
        self.__timeout = 60

    def connect(self):
        print(f"connecting with {self.__timeout}s")


connector = Connector("postgresql://localhost")
connector.connect()
connector.__timeout

postgresql://localhost
60
{'source': 'postgresql://localhost', '_timeout': 60}
postgresql://localhost


AttributeError: 'Connector' object has no attribute '__timeout'

In [46]:
##############################################################################
class Connector:
    def __init__(self, source) -> None:
        self.source = source
        self._timeout = 60


connector = Connector("postgresql://localhost")
print(vars(connector))


##############################################################################
class Connector:
    def __init__(self, source) -> None:
        self.source = source
        self.__timeout = 60


connector = Connector("postgresql://localhost")
print(vars(connector))
print(connector._Connector__timeout)

{'source': 'postgresql://localhost', '_timeout': 60}
{'source': 'postgresql://localhost', '_Connector__timeout': 60}


In [102]:
# property
class Coordinate:
    def __init__(self, lat: float, long: float) -> None:
        self._latitude = self._longitude = None
        self.latitude = lat
        self.longitude = long

    @property
    def latitude(self) -> float:
        return self._latitude

    @latitude.setter
    def latitude(self, lat_value: float) -> None:
        if lat_value not in range(-90, 90 + 1):
            raise ValueError(f"유효하지 않은 위도 값 : {lat_value}")
        self._latitude = lat_value

    @property
    def longitude(self) -> float:
        return self._longitude

    @longitude.setter
    def longitude(self, long_value: float) -> None:
        if long_value not in range(-180, 180 + 1):
            raise ValueError(f"유효하지 않은 경도 값 : {long_value}")
        self._longitude = long_value

In [105]:
coordinate = Coordinate(1, 4)
coordinate.__dict__

{'_latitude': 1, '_longitude': 4}

In [129]:
from typing import List
from dataclasses import dataclass, field

R = 26


@dataclass
class RTrieNode:
    size = R
    value: int
    next_: List["RTrieNode"] = field(default_factory=lambda x: [None] * R)

    def __post_init__(self):
        if len(self.next_) != self.size:
            raise ValueError(f"리스트 (next_)의 길이가 유효하지 않음")


rtrienode = RTrieNode(3, [i for i in range(R)])

In [187]:
from datetime import date
from datetime import timedelta


class DateRangeIterable:
    """자체 이터레이터 메서드를 가지고 있는 이터러블"""

    def __init__(self, start_date, end_date) -> None:
        self.start_date = start_date
        self.end_date = end_date
        self._present_day = start_date

    def __iter__(self):
        return self

    def __next__(self):
        if self._present_day >= self.end_date:
            raise StopIteration
        today = self._present_day
        self._present_day += timedelta(days=1)
        return today

    def a(self):
        return self


for day in DateRangeIterable(date(2023, 1, 1), date(2023, 1, 10)):
    print(day)

2023-01-01
2023-01-02
2023-01-03
2023-01-04
2023-01-05
2023-01-06
2023-01-07
2023-01-08
2023-01-09


In [188]:
r1 = DateRangeIterable(date(2023, 1, 1), date(2023, 1, 3))

In [189]:
next(r1)

datetime.date(2023, 1, 1)

In [169]:
next(r1)

datetime.date(2023, 1, 2)

In [170]:
next(r1)

StopIteration: 

In [191]:
from datetime import date
from datetime import timedelta


class DateRangeContainerIterable:
    """자체 이터레이터 메서드를 가지고 있는 이터러블"""

    def __init__(self, start_date, end_date) -> None:
        self.start_date = start_date
        self.end_date = end_date

    def __iter__(self):
        current_day = self.start_date
        while current_day < self.end_date:
            yield current_day
            current_day += timedelta(days=1)

In [192]:
r1 = DateRangeIterable(date(2023, 1, 1), date(2023, 1, 3))

In [1]:
# 컨테이너


##############################################################################
def mark_coordinate(grid, coord):
    MARKED = 1
    if 0 <= coord.x < grid.width and 0 <= coord.y < grid.height:
        grid[coord] = MARKED


##############################################################################
class Boundaries:
    def __init__(self, width, height) -> None:
        self.width = width
        self.height = height

    def __contains__(self, coord):
        x, y = coord
        return 0 <= x < self.width and 0 <= y < self.height


class Grid:
    def __init__(self, width, height) -> None:
        self.width = width
        self.height = height
        self.boundaries = Boundaries(width, height)

    def __contains__(self, coord):
        return coord in self.boundaries


def mark_coordinate(grid, coord):
    MARKED = 1
    if coord in grid:
        grid[coord] = MARKED

In [6]:
# dynamic attribute
class DynamicAttributes:
    def __init__(self, attribute) -> None:
        self.attribute = attribute

    def __getattr__(self, attr):
        if attr.startswith("fallback_"):
            name = attr.replace("fallback_", "")
            return f"[fallback resolved] {name}"
        raise AttributeError(f"{self.__class__.__name__}에는 {attr} 속성이 없음")


dyn = DynamicAttributes("value")

'value'

In [7]:
dyn.attribute

'value'

In [13]:
dyn.fallback_test

'[fallback resolved] test'

In [16]:
dyn.__dict__["fallback_new"] = "new value"
dyn.fallback_new

'new value'

In [17]:
getattr(dyn, "somthing", "default")

'default'

In [20]:
dyn.get

'value'

In [22]:
def user_display(user_meta: dict = None):
    user_metadata = user_meta or {"name": "a", "age": 1}

    name = user_metadata.pop("name")
    age = user_metadata.pop("age")
    return f"name : {name}, age : {age}"

In [29]:
user_display()

'name : a, age : 1'

In [ ]:
# for custom built in types
from collections import UserDict, UserList, UserString